In [ ]:
import json

import numpy as np
import pandas as pd
import scrapbook as sb
from IPython.display import Markdown as md
from stravalib import Client
from sweat.hrm.heartrate_models import heartrate_model, _heartrate_model_predict

In [ ]:
access_token = None
activity_id = None

In [ ]:
client = Client(access_token)

# Retrieve Strava athlete
athlete_response = client.get_athlete()
athlete = athlete_response.to_dict()
sb.glue('athlete', athlete)

# Retrieve Strava activity details
activity_detail_response = client.get_activity(activity_id)
activity_detail = activity_detail_response.to_dict()
sb.glue('activity_detail', activity_detail)

# Retrieve Strava activity streams
STRAVA_STREAM_TYPES = [
    'time', 'latlng', 'distance', 'altitude', 'velocity_smooth', 'heartrate',
    'cadence', 'watts', 'temp', 'moving', 'grade_smooth'
]
activity_streams_response = client.get_activity_streams(
    activity_id=activity_id,
    types=STRAVA_STREAM_TYPES,
    series_type='time')
activity_streams = {key: val.data for key, val in activity_streams_response.items()}

In [ ]:
md((
    f'# Report for activity: {activity_detail["name"]}\n'
    f'Date: {activity_detail["start_date_local"]}\n\n'
    f'Available data: {", ".join(list(activity_streams.keys()))}'
))

In [ ]:
for key in activity_streams.keys():
    if key in ('time', 'latlng', 'moving'):
        continue
    
    df = pd.DataFrame({'time': activity_streams['time'], key: activity_streams[key]})
    df.plot(x='time', y=key, figsize=(10, 5))
    

## Heart rate model
This model is fitted on your heartrate and power, if available.

In [ ]:
heartrate = activity_streams['heartrate']
power = activity_streams['watts']

# Replace None values with 0
heartrate = [i if isinstance(i, int) else 0 for i in heartrate]
power = [i if isinstance(i, int) else 0 for i in power]

model, _ = heartrate_model(np.array(heartrate), np.array(power))
model

In [ ]:
artificial_power = np.array(120*[100] + 120*[200] + 120*[100])
artificial_heartrate = _heartrate_model_predict(model.params, artificial_power)

In [ ]:
df = pd.DataFrame(dict(power=artificial_power, heartrate=artificial_heartrate))
df.plot(figsize=(15, 10), ylim=(0, 250), yticks=range(0, 250, 10), grid=True);